# Counting Sheep, Courting Sleep:

## Using Python Data Analysis Tools to Find the Perfect Recipe for a Good Night's Sleep

### Step One: Reading and Cleaning

Because I am the creator and owner of the data for this project, and I compiled it by converting a Google Sheet to a CSV file, I was able to clean up the column names while still in the Sheet. This seemed an easier solution than converting via pandas after reading in the data. Google Forms outputs to its associated Google Sheet with the column names being the full question asked in that cell on the Form. I changed these to simple titles in all lower case with words separated by underscores to make them more Pythonic. For reference, below are screen shots of the Sheet before and after the name cleanup.

First, we will import the pandas library to help us manipulate our data.

In [39]:
import pandas as pd

Then we will use pandas to read in our sleep tracking CSV file and, to make sure the data came in correctly, we will check the first five rows.

In [40]:
sleep_df = pd.read_csv('sleep_data_tracking_2023.csv')
sleep_df.head()

,Timestamp,sleep_date,day_of_the_week,hours_sleep,awake_pct,rem_pct,core_pct,deep_pct,hr_bpm_min,hr_bpm_max,resp_rate_min,resp_rate_max,melatonin,magnesium,neuriva,chamomile,bath,shower,meditation
0,2/8/2023 9:10:02,2/7/2023,Tuesday,7.30,3.0,12.0,80.0,5.0,61.0,69.0,8.0,12.5,No,Yes,No,No,Yes,No,Yes
1,2/9/2023 7:56:47,2/8/2023,Wednesday,6.83,5.0,22.0,67.0,6.0,62.0,78.0,8.0,12.5,Yes,Yes,No,No,No,No,Yes
2,2/10/2023 12:10:24,2/9/2023,Thursday,6.68,4.0,11.0,79.0,6.0,61.0,73.0,8.5,12.5,Yes,Yes,No,No,Yes,No,No
3,2/12/2023 17:32:05,2/10/2023,Friday,7.50,4.0,14.0,78.0,4.0,59.0,70.0,9.0,14.0,No,Yes,No,No,No,No,Yes
4,2/12/2023 17:34:12,2/11/2023,Saturday,6.55,3.0,21.0,71.0,5.0,58.0,76.0,8.0,15.0,No,No,No,No,No,No,No


Looks good! Now let's get a feel for our DataFrame and the data types it contains.

In [41]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Timestamp        31 non-null     object 
 1   sleep_date       31 non-null     object 
 2   day_of_the_week  31 non-null     object 
 3   hours_sleep      31 non-null     float64
 4   awake_pct        30 non-null     float64
 5   rem_pct          30 non-null     float64
 6   core_pct         30 non-null     float64
 7   deep_pct         30 non-null     float64
 8   hr_bpm_min       30 non-null     float64
 9   hr_bpm_max       30 non-null     float64
 10  resp_rate_min    30 non-null     float64
 11  resp_rate_max    30 non-null     float64
 12  melatonin        31 non-null     object 
 13  magnesium        31 non-null     object 
 14  neuriva          31 non-null     object 
 15  chamomile        31 non-null     object 
 16  bath             31 non-null     object 
 17  shower           3

We will be dealing with floats and strings in this DataFrame. Now let's get a count of the number of rows and columns.

In [42]:
sleep_df.shape

(31, 19)

Our data set has 31 rows and 19 columns. Quickly comparing the number of rows to the number of non-null values given in the output of cell 3 above, our data is pretty clean in terms of null values. We will have a little cleanup to do with those later.

The "Timestamp" column came in from our Google Sheet because that can't be deleted in Sheets; it can only be hidden. That information is not needed for our analysis; that's simply the time each data submission went into the Google Form. Therefore, that column can be deleted.

In [43]:
del sleep_df['Timestamp']

Let's check our first five rows to make sure that worked.

In [44]:
sleep_df.head()

,sleep_date,day_of_the_week,hours_sleep,awake_pct,rem_pct,core_pct,deep_pct,hr_bpm_min,hr_bpm_max,resp_rate_min,resp_rate_max,melatonin,magnesium,neuriva,chamomile,bath,shower,meditation
0,2/7/2023,Tuesday,7.30,3.0,12.0,80.0,5.0,61.0,69.0,8.0,12.5,No,Yes,No,No,Yes,No,Yes
1,2/8/2023,Wednesday,6.83,5.0,22.0,67.0,6.0,62.0,78.0,8.0,12.5,Yes,Yes,No,No,No,No,Yes
2,2/9/2023,Thursday,6.68,4.0,11.0,79.0,6.0,61.0,73.0,8.5,12.5,Yes,Yes,No,No,Yes,No,No
3,2/10/2023,Friday,7.50,4.0,14.0,78.0,4.0,59.0,70.0,9.0,14.0,No,Yes,No,No,No,No,Yes
4,2/11/2023,Saturday,6.55,3.0,21.0,71.0,5.0,58.0,76.0,8.0,15.0,No,No,No,No,No,No,No


And now let's get a new count of the number of rows and columns.

In [45]:
sleep_df.shape

(31, 18)

Because this data came from daily Google Form submissions, human errors in submitting were probably made. (Especially since the subject is sleep-deprived enough to want to study her own sleep.) It's easy to accidentally send duplicate responses, and when that happens, the respondent can't fix it on their own. Only the Form owner can. So let's see if we have any duplicate entries.

In [46]:
dups = sleep_df[sleep_df.duplicated('sleep_date', keep=False)]
print(dups)

   sleep_date day_of_the_week  hours_sleep  awake_pct  rem_pct  core_pct  \
16  2/23/2023        Thursday         6.48        2.0     17.0      74.0   
17  2/23/2023        Thursday         6.53        4.0     20.0      74.0   

    deep_pct  hr_bpm_min  hr_bpm_max  resp_rate_min  resp_rate_max melatonin  \
16       7.0        59.0        73.0            8.5           13.0        No   
17       2.0        62.0        77.0            9.0           14.5        No   

   magnesium neuriva chamomile bath shower meditation  
16       Yes     Yes        No  Yes     No        Yes  
17       Yes     Yes        No  Yes     No        Yes  


Hmm, it looks as though there's a duplicate entry for 2/23, but not all the data is a duplicate. Let's look at the rows before and after these values at index 16 and 17 to see if there is anything we should delete.

In [47]:
dup_rows = sleep_df.iloc[15:19]
print(dup_rows)

   sleep_date day_of_the_week  hours_sleep  awake_pct  rem_pct  core_pct  \
15  2/22/2023       Wednesday         6.48        2.0     17.0      74.0   
16  2/23/2023        Thursday         6.48        2.0     17.0      74.0   
17  2/23/2023        Thursday         6.53        4.0     20.0      74.0   
18  2/24/2023          Friday         7.40        4.0     10.0      79.0   

    deep_pct  hr_bpm_min  hr_bpm_max  resp_rate_min  resp_rate_max melatonin  \
15       7.0        59.0        73.0            8.5           13.0        No   
16       7.0        59.0        73.0            8.5           13.0        No   
17       2.0        62.0        77.0            9.0           14.5        No   
18       7.0        60.0        70.0            9.0           13.0        No   

   magnesium neuriva chamomile bath shower meditation  
15       Yes     Yes        No   No    Yes        Yes  
16       Yes     Yes        No  Yes     No        Yes  
17       Yes     Yes        No  Yes     No        

It looks like the sleep respondent accidentally submitted the same data in indexes 15 and 16 but with different dates. We know that we can delete the row at index 16. 

In [48]:
sleep_df.drop([16], inplace=True)

Now, let's check the rows in that area to make sure we dropped what we wanted to drop.

In [49]:
dup_rows = sleep_df.iloc[14:19]
print(dup_rows)

   sleep_date day_of_the_week  hours_sleep  awake_pct  rem_pct  core_pct  \
14  2/21/2023         Tuesday         6.87        2.0     16.0      73.0   
15  2/22/2023       Wednesday         6.48        2.0     17.0      74.0   
17  2/23/2023        Thursday         6.53        4.0     20.0      74.0   
18  2/24/2023          Friday         7.40        4.0     10.0      79.0   
19  2/25/2023        Saturday         6.77        4.0      9.0      83.0   

    deep_pct  hr_bpm_min  hr_bpm_max  resp_rate_min  resp_rate_max melatonin  \
14       9.0        63.0        75.0            9.0           12.5        No   
15       7.0        59.0        73.0            8.5           13.0        No   
17       2.0        62.0        77.0            9.0           14.5        No   
18       7.0        60.0        70.0            9.0           13.0        No   
19       4.0        62.0        76.0            9.0           13.5       Yes   

   magnesium neuriva chamomile bath shower meditation  
14    

Looks good. 
We know we have a small number of null values from the numbers output in cell 3. If there was a night where the respondent's Watch did not record sleep data, that could cause a row with multiple null values. For our purposes, an occasional isolated null value won't have a great impact on the overall calcuations and correlations. Those could be filled with the column mean. But if one row (one night's sleep) is missing multiple metrics, it can be deleted from the DataFrame for ease of computation and more accurate data findings. 

In [50]:
sleep_df.isnull().sum()

sleep_date         0
day_of_the_week    0
hours_sleep        0
awake_pct          1
rem_pct            1
core_pct           1
deep_pct           1
hr_bpm_min         1
hr_bpm_max         1
resp_rate_min      1
resp_rate_max      1
melatonin          0
magnesium          0
neuriva            0
chamomile          0
bath               0
shower             0
meditation         0
dtype: int64

We have 1 null value in each of these columns: awake_pct, rem_pct, core_pct, deep_pct, hr_bpm_min, hr_bpm_max, resp_rate_min, and resp_rate_max. This is all Watch data and not subject-reported data, so this could be from the same row. We don't want to assume this and delete these null values, however. Let's take a look at where these null values are. 

In [51]:
null_values = sleep_df[sleep_df.isnull().any(axis=1)]
print(null_values)

  sleep_date day_of_the_week  hours_sleep  awake_pct  rem_pct  core_pct  \
8  2/15/2023       Wednesday          6.5        NaN      NaN       NaN   

   deep_pct  hr_bpm_min  hr_bpm_max  resp_rate_min  resp_rate_max melatonin  \
8       NaN         NaN         NaN            NaN            NaN        No   

  magnesium neuriva chamomile bath shower meditation  
8       Yes     Yes        No   No    Yes        Yes  


As suspected, the null values are all from one row. We have enough entries to delete this one row and still have a representative data set.

In [52]:
sleep_df.drop([8], inplace = True)

Let's check the number of rows we have now.

In [53]:
sleep_df.shape

(29, 18)

That gives us close enough to our goal of one month of sleep data, and with all 29 records now being clean, we're set. 
One last thing before we start exploring relationships in our data--our planned analysis of this data for this project doesn't include any kind of calculations using the "sleep_date" column. But if this project were to be expanded, calculations based on the season or even whether the sleep occurred during EST or DST could need to be performed. Based on the output of cell 3, we can see the sleep_date column is a string. Let's make that a date data type. 

In [54]:
sleep_df['sleep_date'] = pd.to_datetime(sleep_df['sleep_date'])
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 30
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   sleep_date       29 non-null     datetime64[ns]
 1   day_of_the_week  29 non-null     object        
 2   hours_sleep      29 non-null     float64       
 3   awake_pct        29 non-null     float64       
 4   rem_pct          29 non-null     float64       
 5   core_pct         29 non-null     float64       
 6   deep_pct         29 non-null     float64       
 7   hr_bpm_min       29 non-null     float64       
 8   hr_bpm_max       29 non-null     float64       
 9   resp_rate_min    29 non-null     float64       
 10  resp_rate_max    29 non-null     float64       
 11  melatonin        29 non-null     object        
 12  magnesium        29 non-null     object        
 13  neuriva          29 non-null     object        
 14  chamomile        29 non-null     object     

Finally, let's save a clean copy of our DataFrame as a new CSV file we can use in other projects.

In [55]:
sleep_df.to_csv('clean_sleep_data.csv')